In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets, FloatSlider, Layout
from IPython.display import display, Markdown
import warnings
from pathlib import Path

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

# Configure FRED API key
def get_fred_api_key():
    """
    Get FRED API key from environment variable or config file.
    Returns the API key as a string.
    """
    # First try environment variable
    api_key = os.getenv('FRED_API_KEY')
    if api_key:
        return api_key
        
    # Then try config file
    config_path = Path.home() / '.fred_api_key'
    if config_path.exists():
        with open(config_path) as f:
            return f.read().strip()
            
    # If no key found, raise informative error
    raise ValueError(
        "FRED API key not found. Either:\n"
        "1. Set FRED_API_KEY environment variable\n" 
        "2. Create ~/.fred_api_key file with your key\n"
        "Get a key at: https://fred.stlouisfed.org/docs/api/api_key.html"
    )

# Error handling decorator
def handle_errors(func):
    """Decorator for handling common errors in GDP analysis functions"""
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except ValueError as e:
            display(Markdown(f"⚠️ **Value Error:** {str(e)}"))
        except TypeError as e:
            display(Markdown(f"⚠️ **Type Error:** Please check input types. {str(e)}"))
        except Exception as e:
            display(Markdown(f"⚠️ **Error:** An unexpected error occurred. {str(e)}"))
    return wrapper

# Initialize widgets with error checking
def create_validated_slider(*, value, min_val, max_val, step, description):
    """Create a slider widget with input validation"""
    if not (min_val <= value <= max_val):
        raise ValueError(f"{description} initial value must be between {min_val} and {max_val}")
    return FloatSlider(
        value=value,
        min=min_val,
        max=max_val,
        step=step,
        description=description,
        style={'description_width': 'initial'},
        continuous_update=False
    )

# Test widget with validation
try:
    slider = create_validated_slider(
        value=5,
        min_val=0,
        max_val=10,
        step=1,
        description="Test:"
    )
    interact(lambda x: x**2, x=slider)
except Exception as e:
    display(Markdown(f"⚠️ **Widget Error:** {str(e)}"))

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 📘 Real vs. Nominal GDP Over Time

When tracking economic output over time, it's crucial to differentiate between **Nominal GDP** (measured in current prices) and **Real GDP** (measured in constant base-year prices). Nominal GDP reflects changes in both the quantity of goods produced and their prices, while Real GDP isolates changes in quantity only.

This simulation demonstrates how inflation causes Nominal GDP to grow faster than Real GDP, illustrating the importance of adjusting for price changes to understand true economic growth.

# 🔗 The Relationship: Growth Rates & Deflator

1.  **Nominal GDP Growth ($g_N$):** The percentage change in the current-dollar value of output.
2.  **Real GDP Growth ($g_R$):** The percentage change in the constant-dollar value of output (quantity changes).
3.  **Inflation Rate ($\pi$):** The percentage change in the overall price level.

These three are related by the following approximation (especially accurate for small growth rates):

$$ g_N \approx g_R + \pi $$
Or, more precisely:
$$ (1 + g_N) = (1 + g_R) (1 + \pi) $$

**GDP Deflator:** A price index that measures the overall level of prices of all goods and services included in GDP relative to a base year.
$$ \text{GDP Deflator}_t = \frac{\text{Nominal GDP}_t}{\text{Real GDP}_t} \times 100 $$
The inflation rate ($\pi$) is the percentage change in the GDP Deflator:
$$ \pi_t \approx \frac{\text{Deflator}_t - \text{Deflator}_{t-1}}{\text{Deflator}_{t-1}} $$

In this simulation:
* We specify the constant `real_growth` rate ($g_R$) and the constant `inflation` rate ($\pi$).
* We calculate the path of Real GDP: $Y_{Real, t} = Y_0 (1 + g_R)^t$.
* We calculate the path of the GDP Deflator (assuming base year=100): $P_t = 100 (1 + \pi)^t$.
* We calculate Nominal GDP using the relationship: $Y_{Nominal, t} = \frac{Y_{Real, t} \times P_t}{100}$.

The plot shows the diverging paths of Nominal and Real GDP and the rising GDP Deflator reflecting the assumed inflation.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

@handle_errors
def validate_gdp_inputs(base_gdp, inflation_rate, real_growth_rate, T):
    """Validate inputs for GDP visualization"""
    if base_gdp <= 0:
        raise ValueError("Base GDP must be positive")
    if inflation_rate < -0.5 or inflation_rate > 2.0:  # Allow reasonable range
        raise ValueError("Inflation rate must be between -50% and 200%")
    if real_growth_rate < -0.5 or real_growth_rate > 2.0:
        raise ValueError("Real growth rate must be between -50% and 200%")
    if T < 1 or T > 100:
        raise ValueError("Time horizon must be between 1 and 100 years")
    return True

@handle_errors
def plot_real_vs_nominal_gdp(
    base_gdp=1000.0,
    inflation_rate=0.03,
    real_growth_rate=0.02,
    T=20
    ):
    """
    Simulates and plots Nominal GDP, Real GDP, and the GDP Deflator over time
    with comprehensive error handling and validation.

    Args:
        base_gdp (float): Initial Real GDP (and Nominal GDP) in the base year (t=0).
        inflation_rate (float): Constant annual inflation rate (pi).
        real_growth_rate (float): Constant annual real GDP growth rate (gR).
        T (int): Time Horizon in years.
    """
    # Validate inputs
    validate_gdp_inputs(base_gdp, inflation_rate, real_growth_rate, T)
    
    try:
        # Ensure T is an integer
        T = int(T)

        # Time array (0 to T)
        years_idx = np.arange(T + 1)
        start_year = 2024 # Example start year
        years_actual = start_year + years_idx

        # Calculate paths safely with error checking
        try:
            # Real GDP Path: Y_Real(t) = Y_Base * (1 + gR)^t
            real_gdp = base_gdp * np.power(1 + real_growth_rate, years_idx)
            
            # GDP Deflator Path: P(t) = P_Base * (1 + pi)^t (Assume P_Base = 100)
            gdp_deflator = 100.0 * np.power(1 + inflation_rate, years_idx)
            
            # Nominal GDP Path: Y_Nominal(t) = Y_Real(t) * P(t) / 100
            nominal_gdp = real_gdp * gdp_deflator / 100.0
            
            # Calculate Nominal Growth Rate (approx = gR + inflation)
            nominal_growth_rate = (1 + real_growth_rate) * (1 + inflation_rate) - 1
            
        except Exception as e:
            raise ValueError(f"Error calculating GDP paths: {str(e)}")

        # --- Plotting ---
        fig, ax1 = plt.subplots(figsize=(10, 6))

        # Plot Nominal and Real GDP on primary axis
        color_nom = 'darkorange'
        color_real = 'forestgreen'
        ax1.plot(years_actual, nominal_gdp, 
                label=f'Nominal GDP (gN ≈ {nominal_growth_rate:.1%})', 
                linewidth=2, color=color_nom, marker='o', markersize=4)
        ax1.plot(years_actual, real_gdp, 
                label=f'Real GDP (gR = {real_growth_rate:.1%})', 
                linewidth=2, color=color_real, marker='s', markersize=4)
        ax1.fill_between(years_actual, real_gdp, nominal_gdp, 
                        color='lightgray', alpha=0.6, label='Inflation Effect')
        
        # Format primary axis
        ax1.set_xlabel('Year')
        ax1.set_ylabel('GDP ($ Billions, Example)', color='black')
        ax1.tick_params(axis='y', labelcolor='black')
        ax1.grid(True, linestyle='--', alpha=0.7)
        ax1.set_ylim(bottom=0)
        ax1.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))

        # Create secondary axis for GDP Deflator
        ax2 = ax1.twinx()
        color_defl = 'crimson'
        ax2.plot(years_actual, gdp_deflator, 
                label=f'GDP Deflator (π = {inflation_rate:.1%})', 
                linewidth=2, color=color_defl, linestyle='--', marker='^', markersize=4)
        ax2.set_ylabel(f'GDP Deflator (Base Year = {start_year}, Index=100)', 
                    color=color_defl)
        ax2.tick_params(axis='y', labelcolor=color_defl)
        ax2.set_ylim(bottom=0)

        # Combine legends
        lines1, labels1 = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

        ax1.set_title('Real vs. Nominal GDP and GDP Deflator Over Time')
        fig.tight_layout()
        plt.show()

        # --- Display Summary with Validation ---
        end_nominal = nominal_gdp[-1]
        end_real = real_gdp[-1]
        end_deflator = gdp_deflator[-1]
        
        # Calculate compound annual growth rates
        nom_cagr = (end_nominal/base_gdp)**(1/T) - 1
        real_cagr = (end_real/base_gdp)**(1/T) - 1
        inflation_cagr = (end_deflator/100)**(1/T) - 1
        
        summary_md = f"""
        ### 📊 Growth Summary:

        * **Assumed Real GDP Growth (gR):** {real_growth_rate:.2%} per year
        * **Assumed Inflation Rate (π):** {inflation_rate:.2%} per year
        * **Resulting Nominal GDP Growth (gN ≈ gR + π):** {nominal_growth_rate:.2%} per year

        **Values at Year {years_actual[-1]} (T={T}):**
        * **Nominal GDP:** ${end_nominal:,.2f}$
        * **Real GDP:** ${end_real:,.2f}$
        * **GDP Deflator:** {end_deflator:.1f} (Base Year {start_year}=100)
        
        **Compound Annual Growth Rates (CAGR):**
        * **Nominal GDP CAGR:** {nom_cagr:.2%}
        * **Real GDP CAGR:** {real_cagr:.2%}
        * **Average Inflation:** {inflation_cagr:.2%}
        
        **Growth Verification:**
        * $(1 + g_N) = (1 + g_R)(1 + \pi)$
        * ${1 + nominal_growth_rate:.3f} = ({1 + real_growth_rate:.3f})({1 + inflation_rate:.3f})$
        """
        display(Markdown(summary_md))

    except Exception as e:
        raise Exception(f"Error in visualization: {str(e)}")


# --- Create Interactive Widgets with Validation ---
try:
    style = {'description_width': 'initial'}
    layout = Layout(width='95%')

    # Create validated sliders
    base_gdp_slider = create_validated_slider(
        value=1000.0,
        min_val=100,
        max_val=10000,
        step=100,
        description='Base Year GDP ($B):'
    )

    inflation_slider = create_validated_slider(
        value=0.03,
        min_val=-0.02,
        max_val=0.10,
        step=0.005,
        description='Inflation Rate (π):'
    )

    growth_slider = create_validated_slider(
        value=0.02,
        min_val=-0.02,
        max_val=0.08,
        step=0.005,
        description='Real Growth Rate (gR):'
    )

    time_slider = IntSlider(
        value=20,
        min=5,
        max=50,
        step=1,
        description='Years (T):',
        style=style,
        layout=layout
    )

    interact(
        plot_real_vs_nominal_gdp,
        base_gdp=base_gdp_slider,
        inflation_rate=inflation_slider,
        real_growth_rate=growth_slider,
        T=time_slider
    )

except Exception as e:
    display(Markdown(f"⚠️ **Widget Error:** {str(e)}"))

interactive(children=(FloatSlider(value=1000.0, description='Base Year GDP ($B):', layout=Layout(width='95%'),…

## 📊 Real-World Analysis: U.S. GDP and Inflation

Let's analyze actual U.S. data to understand the relationship between nominal GDP, real GDP, and inflation. We'll use data from FRED (Federal Reserve Economic Data) to:

1. Compare historical nominal and real GDP trends
2. Analyze inflation patterns using different price indices
3. Study the relationship between GDP growth and inflation
4. Examine periods of high and low inflation
5. Compare different measures of inflation (GDP Deflator vs CPI)

In [ ]:
@handle_errors
def initialize_fred():
    """Initialize FRED API with proper error handling"""
    try:
        api_key = get_fred_api_key()
        from fredapi import Fred
        fred = Fred(api_key=api_key)
        # Test the connection
        fred.get_series_info('GDP')
        return fred
    except ImportError:
        raise ImportError(
            "fredapi package not installed. Install with:\n"
            "pip install fredapi"
        )
    except Exception as e:
        raise Exception(f"FRED API initialization failed: {str(e)}")

class RealWorldGDPAnalyzer:
    """
    A class for analyzing real-world GDP data using FRED API.
    Includes comprehensive error handling and data validation.
    """
    
    def __init__(self):
        """Initialize the GDP Analyzer with FRED data"""
        self.fred = initialize_fred()
        
        # FRED Series IDs with metadata
        self.series = {
            'GDPC1': {'id': 'GDPC1', 'name': 'Real GDP', 'units': 'Billions of 2017 Dollars'},
            'GDP': {'id': 'GDP', 'name': 'Nominal GDP', 'units': 'Billions of Dollars'},
            'GDPDEF': {'id': 'GDPDEF', 'name': 'GDP Deflator', 'units': 'Index 2017=100'},
            'CPIAUCSL': {'id': 'CPIAUCSL', 'name': 'Consumer Price Index', 'units': 'Index 1982-84=100'},
            'PCEPI': {'id': 'PCEPI', 'name': 'PCE Price Index', 'units': 'Index 2017=100'},
            'M2SL': {'id': 'M2SL', 'name': 'M2 Money Supply', 'units': 'Billions of Dollars'},
            'UNRATE': {'id': 'UNRATE', 'name': 'Unemployment Rate', 'units': 'Percent'}
        }
        
    @handle_errors
    def fetch_data(self, start_date='1947-01-01'):
        """
        Fetch and process data from FRED with error handling
        
        Args:
            start_date (str): Start date in YYYY-MM-DD format
        """
        # Validate start date
        try:
            pd.to_datetime(start_date)
        except Exception:
            raise ValueError(f"Invalid start date format: {start_date}. Use YYYY-MM-DD format.")
            
        # Fetch data with progress tracking
        data = {}
        failed_series = []
        
        display(Markdown("🔄 Fetching data from FRED..."))
        for series_id, info in self.series.items():
            try:
                series_data = self.fred.get_series(info['id'], start_date)
                data[info['name']] = series_data
                display(Markdown(f"✅ Successfully fetched {info['name']}"))
            except Exception as e:
                failed_series.append((info['name'], str(e)))
                display(Markdown(f"❌ Failed to fetch {info['name']}: {str(e)}"))
                
        if failed_series:
            warnings.warn(f"Failed to fetch some series: {failed_series}")
            
        if not data:
            raise Exception("Failed to fetch any data series")
            
        # Process data
        try:
            self.data = pd.DataFrame(data)
            self.data.index = pd.to_datetime(self.data.index)
            
            # Calculate growth rates
            for col in ['Real GDP', 'Nominal GDP', 'GDP Deflator', 'Consumer Price Index', 
                       'PCE Price Index', 'M2 Money Supply']:
                self.data[f'{col} Growth'] = self.data[col].pct_change(4) * 100  # Annual growth
                
            # Calculate output gap using HP filter
            cycle, trend = sm.tsa.filters.hpfilter(np.log(self.data['Real GDP']), lamb=1600)
            self.data['Output Gap'] = cycle * 100  # Convert to percentage
            
            display(Markdown("✅ Data processing completed successfully"))
            
        except Exception as e:
            raise Exception(f"Error processing data: {str(e)}")
        
    @handle_errors
    def validate_plot_inputs(self):
        """Validate data availability for plotting"""
        if self.data is None:
            raise ValueError("No data available. Call fetch_data() first.")
            
        required_cols = [
            'Real GDP', 'Nominal GDP', 'GDP Deflator', 
            'Consumer Price Index', 'PCE Price Index', 
            'M2 Money Supply', 'Unemployment Rate'
        ]
        
        missing_cols = [col for col in required_cols if col not in self.data.columns]
        if missing_cols:
            raise ValueError(f"Missing required data columns: {missing_cols}")
            
    @handle_errors
    def plot_historical_analysis(self):
        """Create comprehensive historical analysis plots with error handling"""
        self.validate_plot_inputs()
        
        try:
            fig = make_subplots(
                rows=3, cols=2,
                subplot_titles=(
                    'Nominal vs Real GDP Growth',
                    'Price Indices',
                    'Phillips Curve Relationship',
                    'Money Supply and Inflation',
                    'Output Gap and Inflation',
                    'Growth-Inflation Scatterplot'
                )
            )
            
            # Plot 1: GDP Growth Rates
            for growth_type in ['Nominal GDP Growth', 'Real GDP Growth']:
                fig.add_trace(
                    go.Scatter(
                        x=self.data.index,
                        y=self.data[growth_type],
                        name=growth_type,
                        hovertemplate="%{y:.1f}% annual growth<br>%{x|%Y-%m}"
                    ),
                    row=1, col=1
                )
            
            # Add recession bands
            for start, end in self.get_nber_recessions():
                if start >= self.data.index[0] and start <= self.data.index[-1]:
                    fig.add_vrect(
                        x0=start,
                        x1=end,
                        fillcolor="gray",
                        opacity=0.2,
                        layer="below",
                        line_width=0,
                        row=1, col=1
                    )
            
            # Plot 2: Price Indices
            for index in ['GDP Deflator Growth', 'Consumer Price Index Growth', 'PCE Price Index Growth']:
                fig.add_trace(
                    go.Scatter(
                        x=self.data.index,
                        y=self.data[index],
                        name=index,
                        hovertemplate="%{y:.1f}% annual growth<br>%{x|%Y-%m}"
                    ),
                    row=1, col=2
                )
            
            # Plot 3: Phillips Curve
            fig.add_trace(
                go.Scatter(
                    x=self.data['Unemployment Rate'],
                    y=self.data['GDP Deflator Growth'],
                    mode='markers',
                    marker=dict(
                        size=6,
                        color=self.data.index.year,
                        colorscale='Viridis',
                        showscale=True,
                        colorbar=dict(title="Year")
                    ),
                    name='Phillips Curve',
                    hovertemplate="Unemployment: %{x:.1f}%<br>Inflation: %{y:.1f}%"
                ),
                row=2, col=1
            )
            
            # Plot 4: Money Supply and Inflation
            fig.add_trace(
                go.Scatter(
                    x=self.data['M2 Money Supply Growth'],
                    y=self.data['GDP Deflator Growth'],
                    mode='markers',
                    marker=dict(
                        size=6,
                        color=self.data.index.year,
                        colorscale='Viridis',
                        showscale=True,
                        colorbar=dict(title="Year")
                    ),
                    name='Money-Inflation',
                    hovertemplate="M2 Growth: %{x:.1f}%<br>Inflation: %{y:.1f}%"
                ),
                row=2, col=2
            )
            
            # Plot 5: Output Gap and Inflation
            fig.add_trace(
                go.Scatter(
                    x=self.data.index,
                    y=self.data['Output Gap'],
                    name='Output Gap',
                    line=dict(color='purple'),
                    hovertemplate="%{y:.1f}% of potential<br>%{x|%Y-%m}"
                ),
                row=3, col=1
            )
            
            fig.add_trace(
                go.Scatter(
                    x=self.data.index,
                    y=self.data['GDP Deflator Growth'],
                    name='Inflation',
                    line=dict(color='red'),
                    hovertemplate="%{y:.1f}% annual rate<br>%{x|%Y-%m}"
                ),
                row=3, col=1
            )
            
            # Plot 6: Growth-Inflation Scatterplot
            fig.add_trace(
                go.Scatter(
                    x=self.data['Real GDP Growth'],
                    y=self.data['GDP Deflator Growth'],
                    mode='markers',
                    marker=dict(
                        size=6,
                        color=self.data.index.year,
                        colorscale='Viridis',
                        showscale=True,
                        colorbar=dict(title="Year")
                    ),
                    name='Growth-Inflation',
                    hovertemplate="Real Growth: %{x:.1f}%<br>Inflation: %{y:.1f}%"
                ),
                row=3, col=2
            )
            
            # Update layout
            fig.update_layout(
                height=1000,
                width=1200,
                showlegend=True,
                title_text="U.S. Historical GDP and Inflation Analysis",
                hovermode='closest'
            )
            
            # Update axes labels
            fig.update_xaxes(title_text="Year", row=1, col=1)
            fig.update_xaxes(title_text="Year", row=1, col=2)
            fig.update_xaxes(title_text="Unemployment Rate (%)", row=2, col=1)
            fig.update_xaxes(title_text="M2 Growth (%)", row=2, col=2)
            fig.update_xaxes(title_text="Year", row=3, col=1)
            fig.update_xaxes(title_text="Real GDP Growth (%)", row=3, col=2)
            
            fig.update_yaxes(title_text="Growth Rate (%)", row=1, col=1)
            fig.update_yaxes(title_text="Growth Rate (%)", row=1, col=2)
            fig.update_yaxes(title_text="Inflation Rate (%)", row=2, col=1)
            fig.update_yaxes(title_text="Inflation Rate (%)", row=2, col=2)
            fig.update_yaxes(title_text="Percent", row=3, col=1)
            fig.update_yaxes(title_text="Inflation Rate (%)", row=3, col=2)
            
            fig.show()
            
            # Display analysis
            self.display_historical_analysis()
            
        except Exception as e:
            raise Exception(f"Error creating visualizations: {str(e)}")
        
    @handle_errors
    def get_nber_recessions(self):
        """Get NBER recession dates with error handling"""
        try:
            # This is a simplified version - you might want to fetch this from FRED
            recessions = [
                ('1969-12-01', '1970-11-01'),
                ('1973-11-01', '1975-03-01'),
                ('1980-01-01', '1980-07-01'),
                ('1981-07-01', '1982-11-01'),
                ('1990-07-01', '1991-03-01'),
                ('2001-03-01', '2001-11-01'),
                ('2007-12-01', '2009-06-01'),
                ('2020-02-01', '2020-04-01')
            ]
            return [(pd.to_datetime(start), pd.to_datetime(end)) 
                    for start, end in recessions]
        except Exception as e:
            warnings.warn(f"Error processing recession dates: {str(e)}")
            return []
        
    @handle_errors
    def display_historical_analysis(self):
        """Display comprehensive analysis of historical patterns"""
        try:
            # Calculate key statistics
            recent_data = self.data.iloc[-4:].mean()  # Last year average
            historical_avg = self.data.mean()
            
            # Calculate correlations
            correlations = self.data[[
                'Real GDP Growth',
                'GDP Deflator Growth',
                'Unemployment Rate',
                'M2 Money Supply Growth',
                'Output Gap'
            ]].corr()
            
            analysis = f"""
            ### 📊 Historical Analysis of U.S. GDP and Inflation
            
            #### 1. Current Economic Conditions (Latest Year Average):
            
            - **Real GDP Growth:** {recent_data['Real GDP Growth']:.1f}%
            - **Nominal GDP Growth:** {recent_data['Nominal GDP Growth']:.1f}%
            - **Inflation Rate:** {recent_data['GDP Deflator Growth']:.1f}%
            - **Unemployment Rate:** {recent_data['Unemployment Rate']:.1f}%
            
            #### 2. Historical Patterns:
            
            1. **Growth-Inflation Relationship:**
               - Correlation: {correlations.loc['Real GDP Growth', 'GDP Deflator Growth']:.2f}
               - Historical avg. real growth: {historical_avg['Real GDP Growth']:.1f}%
               - Historical avg. inflation: {historical_avg['GDP Deflator Growth']:.1f}%
            
            2. **Phillips Curve:**
               - Unemployment-Inflation correlation: {correlations.loc['Unemployment Rate', 'GDP Deflator Growth']:.2f}
               - {'Traditional relationship holds' if correlations.loc['Unemployment Rate', 'GDP Deflator Growth'] < -0.3 else 'Weak relationship'}
            
            3. **Money-Inflation Link:**
               - Correlation: {correlations.loc['M2 Money Supply Growth', 'GDP Deflator Growth']:.2f}
               - {'Strong monetary influence' if correlations.loc['M2 Money Supply Growth', 'GDP Deflator Growth'] > 0.5 else 'Weak monetary influence'}
            
            #### 3. Key Observations:
            
            1. **Growth Dynamics:**
               - Average nominal-real growth gap: {(historical_avg['Nominal GDP Growth'] - historical_avg['Real GDP Growth']):.1f}%
               - Current output gap: {recent_data['Output Gap']:.1f}%
            
            2. **Inflation Measures:**
               - GDP Deflator vs CPI correlation: {self.data['GDP Deflator Growth'].corr(self.data['Consumer Price Index Growth']):.2f}
               - PCE vs CPI correlation: {self.data['PCE Price Index Growth'].corr(self.data['Consumer Price Index Growth']):.2f}
            
            3. **Policy Implications:**
               - {'Inflationary pressures present' if recent_data['GDP Deflator Growth'] > historical_avg['GDP Deflator Growth'] else 'Inflation under control'}
               - {'Economy above potential' if recent_data['Output Gap'] > 0 else 'Economy below potential'}
               - {'Tight labor market' if recent_data['Unemployment Rate'] < historical_avg['Unemployment Rate'] else 'Labor market slack'}
            """
            
            display(Markdown(analysis))
            
        except Exception as e:
            raise Exception(f"Error generating analysis: {str(e)}")

# Initialize and use the analyzer
try:
    real_world_analyzer = RealWorldGDPAnalyzer()
    real_world_analyzer.fetch_data()
    real_world_analyzer.plot_historical_analysis()
except Exception as e:
    display(Markdown(f"⚠️ **Analysis Error:** {str(e)}"))

## 📝 Exercises and Problems

### Exercise 1: Basic Calculations

1. Given the following data for an economy:
   * Year 1: Nominal GDP = $100 billion, GDP Deflator = 100
   * Year 2: Nominal GDP = $120 billion, GDP Deflator = 110
   
   Calculate:
   a) Real GDP for both years
   b) Growth rates of nominal GDP, real GDP, and prices
   c) Verify the relationship between nominal growth, real growth, and inflation

2. The base year changes:
   * Now use Year 2 as the base year (Deflator = 100)
   * Recalculate all values
   * Explain why real GDP levels changed but growth rates didn't

### Exercise 2: Real-World Analysis

Using the RealWorldGDPAnalyzer:

1. **Growth Decomposition:**
   * Analyze periods of high nominal growth
   * Decompose into real growth and inflation
   * Identify which component dominated when

2. **Inflation Measures:**
   * Compare GDP Deflator with CPI
   * Explain why they differ
   * Which is more appropriate for different uses?

3. **Business Cycles:**
   * Identify recession periods
   * Compare nominal and real declines
   * Analyze price behavior during recessions

### Exercise 3: Advanced Topics

1. **Chain-Weighted GDP:**
   * Why was it introduced?
   * How does it differ from fixed-weight measures?
   * What problems does it solve?

2. **Quality Adjustments:**
   * How do price indices account for quality changes?
   * What are the challenges?
   * How might this affect real GDP measurement?

### Research Project Ideas

1. **Historical Analysis:**
   * Choose a significant period (e.g., 1970s stagflation)
   * Analyze nominal vs real GDP behavior
   * Examine policy responses and their effects

2. **International Comparison:**
   * Compare GDP deflators across countries
   * Analyze differences in measurement methods
   * Study the impact on international comparisons

3. **Modern Challenges:**
   * Digital economy measurement issues
   * Quality adjustment in services
   * Impact of globalization on price measurement

### Coding Challenges

1. Enhance the RealWorldGDPAnalyzer:
   * Add new visualizations
   * Include more economic variables
   * Create interactive features

2. Build a GDP Calculator:
   * Create functions for common calculations
   * Add error checking
   * Include different base year options

### Discussion Questions

1. Measurement Issues:
   * What are the main challenges in measuring real GDP?
   * How do statistical agencies handle quality improvements?
   * What might be missing from conventional measures?

2. Policy Implications:
   * How do policymakers use real vs nominal GDP?
   * What are the implications of measurement error?
   * How should policy respond to different types of growth?

3. Modern Economy:
   * How well do our measures capture the digital economy?
   * What improvements might be needed?
   * How might measurement change in the future?

### Data Projects

1. **Historical Analysis:**
   * Download long-run GDP data
   * Create visualizations
   * Identify important patterns
   * Write up your findings

2. **Sectoral Analysis:**
   * Compare price changes across sectors
   * Analyze real vs nominal growth by sector
   * Examine structural changes over time

### Solution Approaches

For Exercise 1:
```python
def calculate_real_gdp(nominal_gdp, deflator, base_year_deflator=100):
    """Calculate real GDP given nominal GDP and deflator"""
    return (nominal_gdp / deflator) * base_year_deflator

def calculate_growth_rates(year1, year2):
    """Calculate growth rates between two years"""
    nominal_growth = (year2['nominal'] / year1['nominal'] - 1) * 100
    real_growth = (year2['real'] / year1['real'] - 1) * 100
    inflation = (year2['deflator'] / year1['deflator'] - 1) * 100
    return nominal_growth, real_growth, inflation
```

For Real-World Analysis:
```python
def analyze_growth_components(data, start_date, end_date):
    """Analyze components of growth over a specific period"""
    period_data = data.loc[start_date:end_date]
    
    nominal_growth = period_data['Nominal GDP Growth'].mean()
    real_growth = period_data['Real GDP Growth'].mean()
    inflation = period_data['GDP Deflator Growth'].mean()
    
    return {
        'nominal_growth': nominal_growth,
        'real_growth': real_growth,
        'inflation': inflation,
        'dominant_factor': 'Real Growth' if abs(real_growth) > abs(inflation) else 'Inflation'
    }
```

In [ ]:
class GDPCalculator:
    def __init__(self):
        """Initialize the GDP calculator"""
        self.data = {}
        
    def add_year(self, year, nominal_gdp, deflator):
        """Add data for a specific year"""
        self.data[year] = {
            'nominal_gdp': nominal_gdp,
            'deflator': deflator
        }
        
    def calculate_real_gdp(self, year, base_year):
        """Calculate real GDP for a given year using specified base year"""
        nominal = self.data[year]['nominal_gdp']
        deflator = self.data[year]['deflator']
        base_deflator = self.data[base_year]['deflator']
        
        return (nominal / deflator) * base_deflator
    
    def calculate_growth_rates(self, year1, year2):
        """Calculate growth rates between two years"""
        # Get data
        nom1 = self.data[year1]['nominal_gdp']
        nom2 = self.data[year2]['nominal_gdp']
        def1 = self.data[year1]['deflator']
        def2 = self.data[year2]['deflator']
        
        # Calculate real GDP for both years (using year1 as base)
        real1 = self.calculate_real_gdp(year1, year1)
        real2 = self.calculate_real_gdp(year2, year1)
        
        # Calculate growth rates
        nominal_growth = ((nom2 / nom1) - 1) * 100
        real_growth = ((real2 / real1) - 1) * 100
        inflation = ((def2 / def1) - 1) * 100
        
        return {
            'nominal_growth': nominal_growth,
            'real_growth': real_growth,
            'inflation': inflation
        }
    
    def analyze_base_year_change(self, old_base, new_base):
        """Analyze the impact of changing the base year"""
        years = sorted(self.data.keys())
        
        # Calculate real GDP series with both base years
        old_base_series = {
            year: self.calculate_real_gdp(year, old_base)
            for year in years
        }
        
        new_base_series = {
            year: self.calculate_real_gdp(year, new_base)
            for year in years
        }
        
        # Create summary table
        summary = []
        for year in years:
            summary.append({
                'Year': year,
                'Nominal_GDP': self.data[year]['nominal_gdp'],
                'Deflator': self.data[year]['deflator'],
                f'Real_GDP_{old_base}_base': old_base_series[year],
                f'Real_GDP_{new_base}_base': new_base_series[year]
            })
        
        return pd.DataFrame(summary)
    
    def display_analysis(self, year1, year2):
        """Display comprehensive analysis"""
        # Calculate real GDP for both years using year1 as base
        real1 = self.calculate_real_gdp(year1, year1)
        real2 = self.calculate_real_gdp(year2, year1)
        
        # Calculate growth rates
        growth_rates = self.calculate_growth_rates(year1, year2)
        
        analysis = f"""
        ### 📊 GDP Analysis: {year1} to {year2}
        
        #### 1. Basic Values
        
        **Year {year1}:**
        - Nominal GDP: ${self.data[year1]['nominal_gdp']:.2f} billion
        - GDP Deflator: {self.data[year1]['deflator']:.1f}
        - Real GDP: ${real1:.2f} billion
        
        **Year {year2}:**
        - Nominal GDP: ${self.data[year2]['nominal_gdp']:.2f} billion
        - GDP Deflator: {self.data[year2]['deflator']:.1f}
        - Real GDP: ${real2:.2f} billion
        
        #### 2. Growth Rates
        
        - Nominal GDP Growth: {growth_rates['nominal_growth']:.1f}%
        - Real GDP Growth: {growth_rates['real_growth']:.1f}%
        - Inflation Rate: {growth_rates['inflation']:.1f}%
        
        #### 3. Relationship Verification
        
        The relationship $(1 + g_N) = (1 + g_R)(1 + \pi)$ implies:
        
        ${(1 + growth_rates['nominal_growth']/100):.3f} \\approx {(1 + growth_rates['real_growth']/100)*(1 + growth_rates['inflation']/100):.3f}$
        
        #### 4. Economic Interpretation
        
        - {'Real growth dominates' if abs(growth_rates['real_growth']) > abs(growth_rates['inflation']) else 'Price changes dominate'}
        - {'Economy grew in real terms' if growth_rates['real_growth'] > 0 else 'Economy contracted in real terms'}
        - {'Experienced inflation' if growth_rates['inflation'] > 0 else 'Experienced deflation'}
        """
        
        display(Markdown(analysis))

# Example solution for Exercise 1
calculator = GDPCalculator()

# Add data
calculator.add_year(1, 100, 100)  # Year 1
calculator.add_year(2, 120, 110)  # Year 2

# Display analysis
calculator.display_analysis(1, 2)

# Show impact of base year change
impact_analysis = calculator.analyze_base_year_change(1, 2)
display(Markdown("\n### 📊 Impact of Base Year Change\n"))
display(impact_analysis)

# Create visualization
plt.figure(figsize=(12, 6))
plt.plot([1, 2], [100, 120], 'b-', label='Nominal GDP', marker='o')
plt.plot([1, 2], [100, calculator.calculate_real_gdp(2, 1)], 'g-', label='Real GDP (Year 1 Base)', marker='s')
plt.plot([1, 2], [calculator.calculate_real_gdp(1, 2), calculator.calculate_real_gdp(2, 2)], 
         'r--', label='Real GDP (Year 2 Base)', marker='^')

plt.title('Nominal and Real GDP with Different Base Years')
plt.xlabel('Year')
plt.ylabel('GDP (billions)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

## 🔄 Chain-Weighted GDP and Modern Challenges

### Chain-Weighted GDP

Chain-weighted GDP was introduced to address the shortcomings of fixed-base GDP calculations. The main advantages are:

1. **Better Handling of Substitution:**
   - Accounts for changing consumption patterns
   - Updates weights regularly
   - More accurate over long periods

2. **Reduced Base Year Dependence:**
   - Uses adjacent-year weights
   - Minimizes distortions from old price structures
   - Better reflects current economic conditions

3. **Technical Implementation:**
   $$\text{Chain-Weighted Growth}_t = \sqrt{\frac{\sum p_{t-1}q_t}{\sum p_{t-1}q_{t-1}} \cdot \frac{\sum p_tq_t}{\sum p_tq_{t-1}}}$$
   where:
   - $p_t$ are prices in year t
   - $q_t$ are quantities in year t

### Modern Measurement Challenges

1. **Digital Economy:**
   - Free digital services
   - Quality improvements in technology
   - Platform economies
   - Cloud services and software

2. **Globalization:**
   - Global value chains
   - Transfer pricing
   - Intellectual property
   - Digital trade

3. **Quality Changes:**
   - Rapid technological improvement
   - New product introduction
   - Service quality
   - Environmental quality

4. **Sharing Economy:**
   - Peer-to-peer services
   - Asset sharing
   - Gig economy
   - Mixed use of personal assets

5. **Intangible Capital:**
   - Research and development
   - Organizational capital
   - Brand value
   - Human capital

### Statistical Approaches

1. **Hedonic Price Adjustment:**
   $$P_{adjusted} = P_{observed} \times f(characteristics)$$
   - Adjusts prices for quality characteristics
   - Used for computers, housing, etc.
   - Complex to implement widely

2. **Quality-Adjusted Price Indices:**
   - Match model methods
   - Overlap pricing
   - Production cost approach
   - User value approach

3. **Digital Service Measurement:**
   - Advertising revenue models
   - User time valuation
   - Opportunity cost approaches
   - Direct utility measurement

### Implications for Policy

1. **Monetary Policy:**
   - Inflation measurement accuracy
   - Real interest rate calculations
   - Policy transmission understanding

2. **Fiscal Policy:**
   - Tax base measurement
   - Public service valuation
   - Infrastructure investment

3. **Structural Policy:**
   - Productivity measurement
   - Innovation policy
   - Competition policy

Let's implement some of these concepts:

In [ ]:
class ChainWeightedGDP:
    """
    A class for analyzing chain-weighted GDP calculations with error handling
    and comprehensive data validation.
    """
    
    def __init__(self):
        """Initialize with sample product data"""
        # Input validation for sample data
        self._validate_product_data({
            'Traditional': {
                2020: {'price': 10, 'quantity': 100},
                2021: {'price': 11, 'quantity': 105},
                2022: {'price': 12, 'quantity': 108},
                2023: {'price': 13, 'quantity': 110},
                2024: {'price': 14, 'quantity': 112}
            },
            'Digital': {
                2020: {'price': 50, 'quantity': 20},
                2021: {'price': 45, 'quantity': 25},
                2022: {'price': 40, 'quantity': 32},
                2023: {'price': 35, 'quantity': 42},
                2024: {'price': 30, 'quantity': 55}
            }
        })
        
    @handle_errors
    def _validate_product_data(self, products):
        """Validate product data structure and values"""
        if not isinstance(products, dict):
            raise ValueError("Products must be a dictionary")
            
        for product, years in products.items():
            if not isinstance(years, dict):
                raise ValueError(f"Years data for {product} must be a dictionary")
                
            for year, data in years.items():
                if not isinstance(year, int):
                    raise ValueError(f"Year {year} must be an integer")
                if not isinstance(data, dict):
                    raise ValueError(f"Data for year {year} must be a dictionary")
                if 'price' not in data or 'quantity' not in data:
                    raise ValueError(f"Missing price or quantity for {product} in {year}")
                if data['price'] <= 0 or data['quantity'] < 0:
                    raise ValueError(f"Invalid price or quantity for {product} in {year}")
                    
        self.products = products
        
    @handle_errors
    def calculate_nominal_gdp(self, year):
        """Calculate nominal GDP for a given year with validation"""
        if year not in self._get_valid_years():
            raise ValueError(f"Invalid year: {year}")
            
        return sum(
            self.products[product][year]['price'] * 
            self.products[product][year]['quantity']
            for product in self.products
        )
    
    @handle_errors
    def calculate_fixed_base_real_gdp(self, year, base_year):
        """Calculate real GDP using fixed base year prices with validation"""
        valid_years = self._get_valid_years()
        if year not in valid_years or base_year not in valid_years:
            raise ValueError(f"Invalid year or base year")
            
        return sum(
            self.products[product][base_year]['price'] * 
            self.products[product][year]['quantity']
            for product in self.products
        )
    
    @handle_errors
    def calculate_chain_weighted_growth(self, year):
        """Calculate chain-weighted GDP growth rate with validation"""
        valid_years = self._get_valid_years()
        if year not in valid_years:
            raise ValueError(f"Invalid year: {year}")
        if year <= min(valid_years):
            return 0.0
        
        try:
            # Calculate Laspeyres quantity index (previous year prices)
            laspeyres = sum(
                self.products[product][year-1]['price'] * 
                self.products[product][year]['quantity']
                for product in self.products
            ) / sum(
                self.products[product][year-1]['price'] * 
                self.products[product][year-1]['quantity']
                for product in self.products
            )
            
            # Calculate Paasche quantity index (current year prices)
            paasche = sum(
                self.products[product][year]['price'] * 
                self.products[product][year]['quantity']
                for product in self.products
            ) / sum(
                self.products[product][year]['price'] * 
                self.products[product][year-1]['quantity']
                for product in self.products
            )
            
            # Fisher ideal index (geometric mean)
            return (np.sqrt(laspeyres * paasche) - 1) * 100
            
        except Exception as e:
            raise ValueError(f"Error calculating chain-weighted growth: {str(e)}")
    
    @handle_errors
    def calculate_chain_weighted_series(self):
        """Calculate chain-weighted GDP index series with validation"""
        years = sorted(self._get_valid_years())
        base_year = years[0]
        
        # Initialize series with base year = 100
        series = {base_year: 100.0}
        
        # Calculate chain-weighted series
        try:
            for year in years[1:]:
                growth_rate = self.calculate_chain_weighted_growth(year)
                series[year] = series[year-1] * (1 + growth_rate/100)
                
            return series
            
        except Exception as e:
            raise ValueError(f"Error calculating chain-weighted series: {str(e)}")
    
    @handle_errors
    def analyze_measurement_approaches(self):
        """Compare different GDP measurement approaches with validation"""
        years = sorted(self._get_valid_years())
        base_year = years[0]
        
        try:
            # Calculate different measures
            results = []
            for year in years:
                nominal = self.calculate_nominal_gdp(year)
                fixed_base = self.calculate_fixed_base_real_gdp(year, base_year)
                chain_weighted = self.calculate_chain_weighted_series()[year]
                
                results.append({
                    'Year': year,
                    'Nominal_GDP': nominal,
                    'Fixed_Base_Real_GDP': fixed_base,
                    'Chain_Weighted_Index': chain_weighted,
                    'Traditional_Price': self.products['Traditional'][year]['price'],
                    'Digital_Price': self.products['Digital'][year]['price'],
                    'Traditional_Quantity': self.products['Traditional'][year]['quantity'],
                    'Digital_Quantity': self.products['Digital'][year]['quantity']
                })
                
            return pd.DataFrame(results)
            
        except Exception as e:
            raise ValueError(f"Error analyzing measurement approaches: {str(e)}")
    
    def _get_valid_years(self):
        """Get the set of valid years across all products"""
        years = set()
        for product in self.products:
            years.update(self.products[product].keys())
        return years
    
    @handle_errors
    def plot_comparison(self):
        """Create visualization comparing different GDP measures with error handling"""
        try:
            data = self.analyze_measurement_approaches()
            
            fig = make_subplots(
                rows=2, cols=2,
                subplot_titles=(
                    'GDP Measures Comparison',
                    'Price Trends',
                    'Quantity Trends',
                    'Growth Rate Comparison'
                )
            )
            
            # Plot 1: GDP Measures
            measures = ['Nominal_GDP', 'Fixed_Base_Real_GDP', 'Chain_Weighted_Index']
            colors = ['blue', 'green', 'red']
            
            for measure, color in zip(measures, colors):
                fig.add_trace(
                    go.Scatter(
                        x=data['Year'],
                        y=data[measure],
                        name=measure,
                        line=dict(color=color),
                        hovertemplate="%{y:.1f}<br>Year: %{x}"
                    ),
                    row=1, col=1
                )
                
            # Plot 2: Price Trends
            fig.add_trace(
                go.Scatter(
                    x=data['Year'],
                    y=data['Traditional_Price'],
                    name='Traditional Price',
                    line=dict(color='purple'),
                    hovertemplate="$%{y:.2f}<br>Year: %{x}"
                ),
                row=1, col=2
            )
            
            fig.add_trace(
                go.Scatter(
                    x=data['Year'],
                    y=data['Digital_Price'],
                    name='Digital Price',
                    line=dict(color='orange'),
                    hovertemplate="$%{y:.2f}<br>Year: %{x}"
                ),
                row=1, col=2
            )
            
            # Plot 3: Quantity Trends
            fig.add_trace(
                go.Scatter(
                    x=data['Year'],
                    y=data['Traditional_Quantity'],
                    name='Traditional Quantity',
                    line=dict(color='brown'),
                    hovertemplate="%{y:.0f} units<br>Year: %{x}"
                ),
                row=2, col=1
            )
            
            fig.add_trace(
                go.Scatter(
                    x=data['Year'],
                    y=data['Digital_Quantity'],
                    name='Digital Quantity',
                    line=dict(color='pink'),
                    hovertemplate="%{y:.0f} units<br>Year: %{x}"
                ),
                row=2, col=1
            )
            
            # Plot 4: Growth Rates
            for measure in measures:
                growth = data[measure].pct_change() * 100
                fig.add_trace(
                    go.Bar(
                        x=data['Year'][1:],
                        y=growth[1:],
                        name=f'{measure} Growth',
                        hovertemplate="%{y:.1f}%<br>Year: %{x}"
                    ),
                    row=2, col=2
                )
                
            # Update layout
            fig.update_layout(
                height=800,
                width=1200,
                title_text="Comparison of GDP Measurement Approaches",
                showlegend=True,
                hovermode='closest'
            )
            
            # Update axes
            fig.update_xaxes(title_text="Year", row=1, col=1)
            fig.update_xaxes(title_text="Year", row=1, col=2)
            fig.update_xaxes(title_text="Year", row=2, col=1)
            fig.update_xaxes(title_text="Year", row=2, col=2)
            
            fig.update_yaxes(title_text="Index/Value", row=1, col=1)
            fig.update_yaxes(title_text="Price ($)", row=1, col=2)
            fig.update_yaxes(title_text="Quantity", row=2, col=1)
            fig.update_yaxes(title_text="Growth Rate (%)", row=2, col=2)
            
            fig.show()
            
            # Display analysis
            self.display_measurement_analysis(data)
            
        except Exception as e:
            raise Exception(f"Error creating visualizations: {str(e)}")
        
    @handle_errors
    def display_measurement_analysis(self, data):
        """Display comprehensive analysis of different measurement approaches"""
        try:
            # Calculate growth rates
            growth_rates = {}
            for measure in ['Nominal_GDP', 'Fixed_Base_Real_GDP', 'Chain_Weighted_Index']:
                growth_rates[measure] = data[measure].pct_change() * 100
                
            # Calculate average growth rates
            avg_growth = {measure: rates.mean() for measure, rates in growth_rates.items()}
            
            # Calculate price and quantity trends
            price_change = {
                'Traditional': (data['Traditional_Price'].iloc[-1] / 
                              data['Traditional_Price'].iloc[0] - 1) * 100,
                'Digital': (data['Digital_Price'].iloc[-1] / 
                           data['Digital_Price'].iloc[0] - 1) * 100
            }
            
            quantity_change = {
                'Traditional': (data['Traditional_Quantity'].iloc[-1] / 
                              data['Traditional_Quantity'].iloc[0] - 1) * 100,
                'Digital': (data['Digital_Quantity'].iloc[-1] / 
                           data['Digital_Quantity'].iloc[0] - 1) * 100
            }
            
            analysis = f"""
            ### 📊 GDP Measurement Analysis
            
            #### 1. Growth Rate Comparison
            
            - **Nominal GDP:** {avg_growth['Nominal_GDP']:.1f}% average growth
            - **Fixed Base Real GDP:** {avg_growth['Fixed_Base_Real_GDP']:.1f}% average growth
            - **Chain-Weighted GDP:** {avg_growth['Chain_Weighted_Index']:.1f}% average growth
            
            #### 2. Price Trends
            
            - **Traditional Product:** {price_change['Traditional']:.1f}% total change
            - **Digital Product:** {price_change['Digital']:.1f}% total change
            - **Implications:** {'Digital deflation present' if price_change['Digital'] < 0 else 'No digital deflation'}
            
            #### 3. Quantity Trends
            
            - **Traditional Product:** {quantity_change['Traditional']:.1f}% total change
            - **Digital Product:** {quantity_change['Digital']:.1f}% total change
            - **Structural Change:** {'Significant digital transformation' if quantity_change['Digital'] > quantity_change['Traditional'] else 'Limited digital transformation'}
            
            #### 4. Measurement Implications
            
            1. **Base Year Effects:**
               - {'Significant' if abs(avg_growth['Fixed_Base_Real_GDP'] - avg_growth['Chain_Weighted_Index']) > 1 else 'Limited'} impact of base year choice
               - Chain-weighted approach {'reduces' if abs(avg_growth['Fixed_Base_Real_GDP'] - avg_growth['Chain_Weighted_Index']) > 1 else 'maintains'} measurement bias
            
            2. **Digital Economy Impact:**
               - Price decline in digital sector: {abs(price_change['Digital']):.1f}%
               - Quantity growth in digital sector: {quantity_change['Digital']:.1f}%
               - {'Significant' if abs(price_change['Digital']) > 20 and quantity_change['Digital'] > 50 else 'Moderate'} measurement challenges
            
            3. **Policy Implications:**
               - {'Inflation might be overstated' if price_change['Digital'] < 0 else 'Inflation measurement accurate'}
               - {'Real growth might be understated' if quantity_change['Digital'] > 50 else 'Real growth measurement accurate'}
               - Need for {'significant' if abs(price_change['Digital']) > 20 else 'moderate'} methodology updates
            """
            
            display(Markdown(analysis))
            
        except Exception as e:
            raise Exception(f"Error generating analysis: {str(e)}")

# Create and use the chain-weighted GDP analyzer
try:
    chain_weighted_analyzer = ChainWeightedGDP()
    chain_weighted_analyzer.plot_comparison()
except Exception as e:
    display(Markdown(f"⚠️ **Analysis Error:** {str(e)}"))

# 📘 Real vs. Nominal GDP Over Time

GDP can be measured in **current prices** (nominal) or **constant prices** (real).  
Real GDP tells us what’s happening to quantities — **not prices**.

# 🧠 Nominal vs. Real

| Nominal GDP | Real GDP |
|-------------|----------|
| Current prices | Constant prices |
| Includes price changes | Strips out inflation |
| Grows faster when prices rise | Shows real output growth |

We convert nominal to real using a **GDP deflator**:
\[
\text{Real GDP} = \frac{\text{Nominal GDP}}{\text{Deflator}} \times 100
\]

# 📈 Why This Matters

Imagine GDP goes up 10% — but prices rose 10%. Did real output increase?  
**No!** All the increase is just inflation. Real GDP stays flat.

> Real GDP allows us to **compare across time**, not just see dollar values.
